In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/siamese_network/5_cls.zip

Archive:  /content/drive/MyDrive/siamese_network/5_cls.zip
  inflating: 5_cls/discription.txt   
   creating: 5_cls/test/
   creating: 5_cls/test/cataract/
  inflating: 5_cls/test/cataract/10_JPG_jpg.rf.71aa847d03aeaac1f45031187a950b8b.jpg  
  inflating: 5_cls/test/cataract/11_JPG_jpg.rf.8067a193cb439e447e052ddb56f25254.jpg  
  inflating: 5_cls/test/cataract/11_JPG_jpg.rf.c8be22ccffefdd94fe30c7037f434d47.jpg  
  inflating: 5_cls/test/cataract/12_jpg.rf.5c479c57e5106609361b54815f637de9.jpg  
  inflating: 5_cls/test/cataract/13_jpg.rf.fdf261a3cb970035f83ae5a0c56f3d05.jpg  
  inflating: 5_cls/test/cataract/13_JPG_jpg.rf.b15332d10dce8ee102ce78ced867023b.jpg  
  inflating: 5_cls/test/cataract/14_jpg.rf.6cd4c8f520cc3e17c4d19edac9b8410a.jpg  
  inflating: 5_cls/test/cataract/14_jpg.rf.706c5a49e2f17f71070907f6b7407e6f.jpg  
  inflating: 5_cls/test/cataract/15_jpg.rf.2e255cf78204e94b49ada7b89a5be7b4.jpg  
  inflating: 5_cls/test/cataract/15_jpg.rf.f3ded639e6a90391ac5ea8f37bc436c4.jpg  
  inflat

In [ ]:
import numpy as np
import cv2
import tensorflow as tf

In [ ]:
train_path = r"/content/5_cls/train"
test_path = r"/content/5_cls/test"
val_path = r"/content/5_cls/val"

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
train_data = train_datagen.flow_from_directory(directory=train_path,target_size=(224,224),shuffle=True,seed=42,batch_size=1205,class_mode='sparse')
test_data = test_datagen.flow_from_directory(directory=test_path,target_size=(224,224),seed=42,batch_size=156,class_mode='sparse')
val_data = val_datagen.flow_from_directory(directory=val_path,target_size=(224,224),seed=42,batch_size=148,class_mode='sparse')

Found 1205 images belonging to 5 classes.
Found 156 images belonging to 5 classes.
Found 148 images belonging to 5 classes.


In [ ]:
train_data = next(iter(train_data))
test_data = next(iter(test_data))
val_data = next(iter(val_data))

In [ ]:


class Image_pair_generator(tf.keras.utils.Sequence):

  def __init__(self,data,input_size = (224,224,3),batch_size = 32):
    self.images,self.labels = data
    self.input_size = input_size
    self.batch_size = batch_size
    self.n = len(self.labels)


  def __len__(self):
    return self.n

  def __getitem__(self,index):
    current_img = self.images[index]
    current_label = self.labels[index]
    image_pairs = []
    pair_label = []

    for _ in range(self.batch_size//2):
      pos_id = np.random.choice(np.where(self.labels == current_label)[0])
      neg_id = np.random.choice(np.where(self.labels != current_label)[0])
      pos_img = self.images[pos_id]
      neg_img = self.images[neg_id]
      image_pairs.append([current_img,pos_img])
      pair_label.append([1])
      image_pairs.append([current_img,neg_img])
      pair_label.append([0])

    return [np.array(image_pairs)[:,0],np.array(image_pairs)[:,1]],np.array(pair_label)

In [ ]:
train_pair = Image_pair_generator(train_data)
test_pair = Image_pair_generator(test_data)
val_pair = Image_pair_generator(val_data)

In [ ]:
test = train_pair.__getitem__(0)

In [ ]:
from keras.models import Model
from keras.layers import Conv2D,Dense,MaxPool2D,Flatten,Dropout,Rescaling,Reshape,Input,GlobalAveragePooling2D,BatchNormalization,MaxPooling2D

In [ ]:
def feature_extractor(input_shape,embedding_dim = 46):
  input = Input(input_shape)
  x = Conv2D(32, kernel_size=(7,7), strides=(1,1), padding='same', activation='relu')(input)
  x = BatchNormalization()(x)
  x = MaxPool2D(pool_size=(2,2))(x)
  x = Conv2D(64, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(pool_size=(2,2))(x)
  x = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(pool_size=(2,2))(x)
  pooled_output = GlobalAveragePooling2D()(x)
  output = Dense(embedding_dim)(pooled_output)
  model = Model(input,output)
  return model

In [ ]:
feature_extractor((224,224,3)).summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      4736      
                                                                 
 batch_normalization (Batch  (None, 224, 224, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      51264     
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 64)      256   

In [ ]:
import keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=1,keepdims=True))

In [ ]:
def siamese_network(input_shape=(224,224,3)):
  image_a = Input(input_shape)
  image_b = Input(input_shape)
  feature = feature_extractor(input_shape)
  feat_a = feature(image_a)
  feat_b = feature(image_b)
  dist = euclidean_distance_loss(feat_a,feat_b)
  outputs = Dense(1,activation='sigmoid')(dist)
  model = Model(inputs=[image_a,image_b],outputs=outputs)
  return model

In [ ]:
siamese_network().summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 model_1 (Functional)        (None, 46)                   136686    ['input_2[0][0]',             
                                                                     'input_3[0][0]']             
                                                                                                  
 tf.math.subtract (TFOpLamb  (None, 46)                   0         ['model_1[1][0]',       

In [ ]:
model = siamese_network()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(train_pair,validation_data=val_pair,epochs=2,batch_size=64)

Epoch 1/2
1205/1205 [==============================] - 227s 188ms/step - loss: 0.0456 - accuracy: 0.9874 - val_loss: 0.8454 - val_accuracy: 0.7804
Epoch 2/2
1205/1205 [==============================] - 224s 185ms/step - loss: 0.0436 - accuracy: 0.9880 - val_loss: 0.9923 - val_accuracy: 0.7489


In [ ]:
model.save("/content/drive/MyDrive/siamese_network/siamese.keras")

In [ ]:
model.evaluate(test_pair)

156/156 [==============================] - 9s 55ms/step - loss: 0.8645 - accuracy: 0.7957


[0.8644895553588867, 0.7956730723381042]